# Session 5 — Prompt Engineering **V4 고급 전략 Ⅱ**

| 버전 | 전략 | 핵심 아이디어 | 해결 문제 |
|------|------|--------------|-----------|
| **V4.1** | **ReAct (Reason + Act)** | 사고와 도구 호출을 단일 프롬프트에 통합 | LLM‑Tool 왕복 ↓ |
| **V4.2** | **Multiple Chains** | Task 분할 & 병렬/분기 실행 | 지연·토큰 낭비 ↓ |
| **V4.3** | **Meta Prompting** | 프롬프트 요약·자기 점검 | 토큰 절약 & 품질 유지 |
| **V4.4** | **Output Length Control** | 응답 길이 명시 제어 | 장황·잘림 방지 |
| **V4.5** | **Max Token Bypass** | 슬라이딩 윈도·요약 후 최종 응답 | 최대 토큰 제한 회피 |
---
---
> **모델:** `gpt‑4o‑mini`  
> **데이터:** `./data/05_session_dataset.csv` (10행 예시)  
> **목표:** minimal V1 → V4.* 전략으로 **응답 / 지연 시간 / 비용** 비교


## 📦 패키지 설치

In [1]:
# VSCODE
# !pip install -r ../requirements.txt
# COLAB
!pip install -r requirements.txt

In [2]:
# Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## ⚙️ 환경 설정 및 Langfuse 초기화

In [3]:
import os, asyncio, time, nest_asyncio, pandas as pd
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
USE_STUB = OPENAI_API_KEY is None

# Langfuse
try:
    from langfuse import Langfuse
    langfuse = Langfuse()
except:
    langfuse = None
    print('⚠️ Langfuse 불러오기 실패')

if not USE_STUB:
    from langfuse.openai import AsyncOpenAI
    client = AsyncOpenAI(api_key=OPENAI_API_KEY)
else:
    client = None
    print('🔧 Stub 모드: OPENAI_API_KEY 없음 → 더미 응답 사용')

nest_asyncio.apply()

PRICE = {'input': 0.15/1_000_000, 'output': 0.60/1_000_000}

async def call_openai(system_p, user_p, tag='V0', chat_history=None):
    start = time.perf_counter_ns()
    if USE_STUB:
        await asyncio.sleep(0.05)
        answer = f'[STUB {tag}] {user_p[:30]}...'
        prompt_tok, completion_tok = 30, 120
    else:
        messages = chat_history.copy() if chat_history else []
        messages += [{'role':'system','content':system_p},
                     {'role':'user','content':user_p}]
        resp = await client.chat.completions.create(model='gpt-4o-mini', messages=messages)
        answer = resp.choices[0].message.content.strip()
        usage = resp.usage
        prompt_tok, completion_tok = usage.prompt_tokens, usage.completion_tokens
    latency = (time.perf_counter_ns() - start) / 1_000_000
    cost = prompt_tok*PRICE['input'] + completion_tok*PRICE['output']
    # if langfuse and not USE_STUB:
    #     langfuse.trace(name=tag, input=user_p, output=answer)
    return dict(answer=answer, latency_ms=latency,
                prompt_tokens=prompt_tok, completion_tokens=completion_tok,
                usd_cost=cost)


## 🗂 데이터 로드

In [4]:
DATA = Path('./data/05_session_dataset.csv')
if DATA.exists():
    df = pd.read_csv(DATA)
else:
    print('⚠️ 파일이 없어 샘플 10행 생성')
    sample = [
        ('order_delivery','주문한 상품이 아직 도착하지 않았어요.'),
        ('refund','환불 요청했는데 진행 상황이 궁금합니다.'),
        ('account_login','계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.')
    ]
    df = pd.DataFrame(sample * 4, columns=['scenario','question']).head(10)
df.head()

⚠️ 파일이 없어 샘플 10행 생성


,scenario,question
0,order_delivery,주문한 상품이 아직 도착하지 않았어요.
1,refund,환불 요청했는데 진행 상황이 궁금합니다.
2,account_login,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다."
3,order_delivery,주문한 상품이 아직 도착하지 않았어요.
4,refund,환불 요청했는데 진행 상황이 궁금합니다.


## 🏃 실행 도우미

In [5]:
async def run(df, version, build_sys, build_user=lambda r: r['question'], **kwargs):
    tasks=[]
    for _, row in df.iterrows():
        tasks.append(call_openai(build_sys(row), build_user(row), tag=version, **kwargs))
    res = await asyncio.gather(*tasks)
    out = df.copy()
    for i, r in enumerate(res):
        for k,v in r.items():
            out.loc[i, f'{version}_{k}'] = v
    return out

## 🔹 Baseline — V1

In [6]:
def sys_v1(row):
    return 'You are a polite Korean customer-service chatbot. Reply in Korean in 5 sentences max.'
baseline = await run(df, 'V1', sys_v1)
baseline[['question', 'V1_answer']]

,question,V1_answer
0,주문한 상품이 아직 도착하지 않았어요.,"안녕하세요! 고객님, 주문하신 상품이 아직 도착하지 않아 불편을 드려 죄송합니다. ..."
1,환불 요청했는데 진행 상황이 궁금합니다.,안녕하세요! 환불 요청에 대한 진행 상황을 확인해 드리겠습니다. 요청하신 환불은 현...
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",안녕하세요! 계정이 잠겼다면 불편을 드려서 죄송합니다. 재설정 메일이 도착하지 않는...
3,주문한 상품이 아직 도착하지 않았어요.,"안녕하세요! 고객님, 주문하신 상품이 아직 도착하지 않아 불편을 드린 점 사과드립니..."
4,환불 요청했는데 진행 상황이 궁금합니다.,"안녕하세요! 환불 요청에 대해 문의주셔서 감사합니다. 현재 환불 처리 중이며, 보통..."
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",안녕하세요! 계정이 잠겼다니 불편을 드려 죄송합니다. 재설정 메일이 오지 않는 경우...
6,주문한 상품이 아직 도착하지 않았어요.,"고객님, 불편을 드려서 죄송합니다. 주문하신 상품의 배송 상태를 확인해 보겠습니다...."
7,환불 요청했는데 진행 상황이 궁금합니다.,안녕하세요! 저희 고객센터에 문의해 주셔서 감사합니다. 환불 요청은 현재 처리 중이...
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",죄송합니다. 계정 잠금과 재설정 메일 문제로 불편을 드려 죄송합니다. 스팸 메일함을...
9,주문한 상품이 아직 도착하지 않았어요.,죄송합니다. 주문하신 상품이 아직 도착하지 않았군요. 주문 상태를 확인해 보겠습니다...


## V4.1 — ReAct

In [7]:
def sys_V4_1(row):
    return "Think step-by-step and if needed say 'ACT:lookup_tracking'. Reply in Korean."
v_V4_1 = await run(baseline, 'V4_1', sys_V4_1)
v_V4_1[['question', 'V4_1_answer']]

,question,V4_1_answer
0,주문한 상품이 아직 도착하지 않았어요.,"주문한 상품이 아직 도착하지 않았다면, 다음과 같은 단계를 시도해 보세요.\n\n1..."
1,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청의 진행 상황을 확인하려면 고객 서비스나 환불 요청이 이루어진 플랫폼에 직...
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.","계정이 잠겼고 재설정 메일이 오지 않는 경우, 다음 단계를 시도해 보세요:\n\n1..."
3,주문한 상품이 아직 도착하지 않았어요.,"주문한 상품이 도착하지 않았다면, 몇 가지 단계를 따라 확인해보세요.\n\n1. *..."
4,환불 요청했는데 진행 상황이 궁금합니다.,"환불 요청 진행 상황을 확인하려면, 해당 거래나 환불 요청에 대한 세부 정보를 알려..."
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.","계정이 잠겼고 재설정 메일이 오지 않는 경우, 다음 단계를 시도해 보세요.\n\n1..."
6,주문한 상품이 아직 도착하지 않았어요.,주문한 상품이 아직 도착하지 않았다면 몇 가지 확인해보아야 할 사항이 있습니다.\n...
7,환불 요청했는데 진행 상황이 궁금합니다.,"환불 요청의 진행 상황을 확인하려면, 해당 업체나 서비스의 고객 지원 팀에 직접 연..."
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",계정이 잠겼고 재설정 메일이 오지 않는 경우 다음 단계를 따라해 보세요:\n\n1....
9,주문한 상품이 아직 도착하지 않았어요.,주문한 상품이 도착하지 않았다면 몇 가지 확인해 볼 사항이 있습니다.\n\n1. *...


## V4.2 — Multiple Chains

In [8]:
def sys_V4_2(row):
    return "Split into Task 1: Identify issue, Task 2: Solve. Combine into final answer in Korean."
v_V4_2 = await run(v_V4_1, 'V4_2', sys_V4_2)
v_V4_2[['question', 'V4_2_answer']]

,question,V4_2_answer
0,주문한 상품이 아직 도착하지 않았어요.,Task 1: 문제 확인 - 주문한 상품이 도착하지 않았습니다.\n\nTask 2:...
1,환불 요청했는데 진행 상황이 궁금합니다.,작업 1: 문제 식별 \n- 환불 요청을 했으나 진행 상황에 대한 정보가 부족하다...
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.","Task 1: Identify issue \n사용자의 계정이 잠겨 있고, 재설정 ..."
3,주문한 상품이 아직 도착하지 않았어요.,Task 1: 주문한 상품이 아직 도착하지 않았다는 문제를 확인했습니다.\n\nTa...
4,환불 요청했는데 진행 상황이 궁금합니다.,작업 1: 환불 요청의 진행 상황을 문의하는 것이 문제입니다. \n작업 2: 고객...
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.","Task 1: 문제 확인하기 \n계정이 잠겼고, 비밀번호 재설정 이메일이 오지 않..."
6,주문한 상품이 아직 도착하지 않았어요.,Task 1: 문제 파악하기\n주문한 상품이 아직 도착하지 않았다는 문제입니다. 배...
7,환불 요청했는데 진행 상황이 궁금합니다.,Task 1: 환불 요청의 진행 상황을 확인해야 하는 이슈입니다.\n\nTask 2...
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",Task 1: 문제 확인\n- 계정이 잠겼다는 메시지와 재설정 메일이 오지 않는 상...
9,주문한 상품이 아직 도착하지 않았어요.,Task 1: 문제 확인 - 주문한 상품이 아직 도착하지 않은 상황입니다.\n\nT...


## V4.3 — Meta Prompting

In [9]:
def sys_V4_3(row):
    return "Summarize question in 12 tokens. Self-check then reply in Korean."
v_V4_3 = await run(v_V4_2, 'V4_3', sys_V4_3)
v_V4_3[['question', 'V4_3_answer']]

,question,V4_3_answer
0,주문한 상품이 아직 도착하지 않았어요.,주문한 상품이 언제 도착하나요?
1,환불 요청했는데 진행 상황이 궁금합니다.,환불 진행 상황을 확인하고 싶습니다.
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",계정 잠금 해제 메일을 받지 못했어요.
3,주문한 상품이 아직 도착하지 않았어요.,주문한 상품의 도착 지연에 대한 문의입니다.
4,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청 진행 상황에 대해 알고 싶습니다.
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",계정 잠금 해제 메일이 오지 않으면 어떻게 하나요?
6,주문한 상품이 아직 도착하지 않았어요.,주문한 상품이 도착하지 않았습니다.
7,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청 진행 상황을 알고 싶습니다.
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",계정 잠김 문제와 재설정 메일 미수신 관련 질문입니다.
9,주문한 상품이 아직 도착하지 않았어요.,상품이 아직 도착하지 않았다고 하셨습니다.


## V4.4 — Output Length Control

In [10]:
def sys_V4_4(row):
    return "Answer in exactly 2 Korean sentences, each ≤20 words."
v_V4_4 = await run(v_V4_3, 'V4_4', sys_V4_4)
v_V4_4[['question', 'V4_4_answer']]

,question,V4_4_answer
0,주문한 상품이 아직 도착하지 않았어요.,고객센터에 문의해 보세요. 배송 조회를 통해 상황을 확인할 수 있습니다.
1,환불 요청했는데 진행 상황이 궁금합니다.,환불 진행 상황을 확인하려면 고객센터에 문의해주세요. 빠른 시일 내에 처리될 것입니다.
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",스팸 폴더를 확인해 보세요. 이메일 주소가 맞는지 다시 한 번 체크해 보시기 바랍니다.
3,주문한 상품이 아직 도착하지 않았어요.,주문 상태를 확인해 보세요. 고객센터에 문의하면 도움이 됩니다.
4,환불 요청했는데 진행 상황이 궁금합니다.,고객 지원팀에 문의하여 진행 상황을 확인하실 수 있습니다. 빠른 처리로 답변을 받을...
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",메일함을 확인하고 스팸 폴더도 확인해 보세요. 여전히 문제면 고객 지원에 문의하세요.
6,주문한 상품이 아직 도착하지 않았어요.,주문하신 상품의 배송 상태를 확인해 보겠습니다. 잠시만 기다려 주세요.
7,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청에 대한 진행 상황을 확인중입니다. 곧 연락드리겠습니다.
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",메일 주소가 정확한지 확인해 보세요. 스팸 폴더도 확인해 보시면 좋습니다.
9,주문한 상품이 아직 도착하지 않았어요.,배송 확인을 위해 주문 번호를 알려주세요. 빠른 확인 후 안내드리겠습니다.


## V4.5 — Max Token Bypass

In [11]:
def sys_V4_5(row):
    return "If long input, summarize parts and return 4-sentence final summary in Korean."
v_V4_5 = await run(v_V4_4, 'V4_5', sys_V4_5)
v_V4_5[['question', 'V4_5_answer']]

,question,V4_5_answer
0,주문한 상품이 아직 도착하지 않았어요.,"고객님께서 주문하신 상품이 아직 도착하지 않았다면, 주문 상태를 확인하거나 배송 추..."
1,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청 후 진행 상황에 대한 문의가 있으시군요. 환불 과정은 일반적으로 요청이 ...
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",계정 잠금 문제로 인해 재설정 메일을 받지 못하고 계신 것 같습니다. 우선 스팸 또...
3,주문한 상품이 아직 도착하지 않았어요.,"주문한 상품이 아직 도착하지 않았다면, 다음과 같은 단계를 고려해보세요. 첫째, 주..."
4,환불 요청했는데 진행 상황이 궁금합니다.,"환불 요청이 진행 중인 경우, 일반적으로 처리 상태를 확인하는 방법은 고객센터에 문..."
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.","계정이 잠겼다면, 일반적으로 재설정 메일을 통해 문제를 해결할 수 있습니다. 하지만..."
6,주문한 상품이 아직 도착하지 않았어요.,상품이 아직 도착하지 않았다는 불편한 상황에 대해 말씀하셨군요. 주문 상태를 확인하...
7,환불 요청했는데 진행 상황이 궁금합니다.,"환불 요청 후 진행 상황을 확인하고자 하는 경우, 고객 지원팀에 문의하거나 요청 확..."
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",계정 잠금 문제로 인해 재설정 메일이 오지 않는 경우가 있습니다. 먼저 스팸 폴더를...
9,주문한 상품이 아직 도착하지 않았어요.,"상품이 도착하지 않은 경우, 주문 상태를 확인하거나 판매자에게 연락해보는 것이 좋습..."


## 📊 비교 — Latency & Cost

In [12]:
cols = ['question']
for v in ['V1','V4_1','V4_2','V4_3','V4_4','V4_5']:
    cols += [f'{v}_latency_ms', f'{v}_usd_cost']
v_V4_5[cols]

,question,V1_latency_ms,V1_usd_cost,V4_1_latency_ms,V4_1_usd_cost,V4_2_latency_ms,V4_2_usd_cost,V4_3_latency_ms,V4_3_usd_cost,V4_4_latency_ms,V4_4_usd_cost,V4_5_latency_ms,V4_5_usd_cost
0,주문한 상품이 아직 도착하지 않았어요.,1433.013815,0.000045,5438.344284,0.000142,1511.088899,0.000054,729.574760,0.000012,673.877133,0.000017,1734.349941,0.000056
1,환불 요청했는데 진행 상황이 궁금합니다.,1669.167869,0.000041,2124.954904,0.000059,2025.215940,0.000072,702.217317,0.000012,746.057235,0.000020,2598.594691,0.000090
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",1478.400374,0.000057,4409.084374,0.000156,3742.137753,0.000126,916.131274,0.000016,655.031049,0.000023,1966.379202,0.000058
3,주문한 상품이 아직 도착하지 않았어요.,1464.409852,0.000045,3542.776215,0.000115,1833.049689,0.000059,540.271535,0.000014,606.125075,0.000016,2377.575670,0.000092
4,환불 요청했는데 진행 상황이 궁금합니다.,1191.129882,0.000038,1471.940704,0.000051,1831.028978,0.000054,489.062466,0.000012,1379.518470,0.000022,3208.386424,0.000116
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",2367.982352,0.000058,4213.470759,0.000137,2719.062676,0.000108,727.420441,0.000017,680.610044,0.000023,3395.418698,0.000082
6,주문한 상품이 아직 도착하지 않았어요.,1000.769578,0.000034,3032.657784,0.000101,1785.508619,0.000082,602.948428,0.000012,1061.156366,0.000018,1989.805369,0.000055
7,환불 요청했는데 진행 상황이 궁금합니다.,1197.344649,0.000040,3481.006205,0.000127,1747.890360,0.000058,487.218166,0.000012,684.402833,0.000017,1502.232895,0.000054
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",2814.106873,0.000064,4198.599942,0.000135,6293.934138,0.000135,1214.539616,0.000018,720.536382,0.000022,1999.781550,0.000061
9,주문한 상품이 아직 도착하지 않았어요.,1114.016215,0.000037,2410.755912,0.000093,1605.006638,0.000065,548.953475,0.000013,708.821462,0.000017,1674.313613,0.000056


---

## 🧾 실험 기록 예시

```markdown
### 사용자 요청
"계정이 잠겼다고 뜨는데, 재설정 링크도 안 와요."

### 🔹 V1 응답 결과
- (기본 Prompt 생성 응답)

### 🔸 V4 전략 적용
- [ ] V4.1 ReAct
- [ ] V4.2 Multiple Chains
- [ ] V4.3 Meta Prompting
- [ ] V4.4 Output Length Control
- [ ] V4.5 Max Token Bypass

### 🔸 개선된 Prompt 설계
(System & User Prompt 조합)

### ✅ 개선된 응답
(적용 전략 기반 응답)

### 🧠 정성 평가
- 응답 간결성, 정확성, 문제 해결 흐름 개선 여부
```

## ✍️ 개인 실습 영역

In [ ]:
# TODO: 아래에 직접 실험 코드를 작성해 보세요.

# 끝 🎉

## 작업한 V4.0 Prompt Langfuse에 등록

In [13]:
from pathlib import Path
from langfuse import Langfuse

def parse_prompty(path: Path):
    """Langfuse-style .prompty → ChatPrompt 형태로 변환"""
    content = path.read_text(encoding="utf-8")
    sections = content.strip().split('---')

    if len(sections) < 3:
        raise ValueError("❌ .prompty 파일은 YAML + system + user prompt 형식이어야 합니다.")

    _ = sections[1]
    prompt_block = sections[2]

    # 각 부분 추출
    system_prompt = ""
    user_prompt = ""
    current_role = None
    lines = prompt_block.strip().splitlines()

    for line in lines:
        if line.strip().startswith("system:"):
            current_role = "system"
            continue
        elif line.strip().startswith("user:"):
            current_role = "user"
            continue

        if current_role == "system":
            system_prompt += line + "\n"
        elif current_role == "user":
            user_prompt += line + "\n"

    print(system_prompt)
    print(user_prompt)

    return [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": user_prompt.strip()}
    ]

# Langfuse Prompt 등록
lf = Langfuse()

# VSCODE
# PROMPT_PATH = Path("../prompts/01_order_delivery/v4_0.prompty")
# COLAB
PROMPT_PATH = Path("/content/drive/MyDrive/KAKAO - 01 - 고급 프롬프트 엔지니어링/kakao_01_advanced_prompt_engineering/prompts/01_order_delivery/v4_0.prompty")

PROMPT_NAME = "order_delivery"
version = "4.0"

chat_prompt = parse_prompty(PROMPT_PATH)

try:
    existing = lf.get_prompt(name=PROMPT_NAME, type="chat")
except Exception as e:
    if "404" in str(e):
        existing = None
    else:
        raise e

if existing:
    lf.update_prompt(
        prompt_id = existing.id,
        prompt    = chat_prompt,
        tags      = ["smart_cs"],
        labels    = ["stable"],
    )
    print("🔄 Prompt updated (v4.0)")
else:
    lf.create_prompt(
        name      = PROMPT_NAME,
        type      = "chat",
        prompt    = chat_prompt,
        tags      = ["smart_cs"],
        labels    = ["stable"],
    )
    print("✅ Prompt created (v4.0)")

print("👀 Langfuse UI ▸ Prompts ▸ order_delivery 확인")


  당신은 **30대 중반 숙련 전자상거래 배송 CS 담당자**입니다.  
  말투는 항상 **차분·전문적**, 고객 감정에 공감부터 제시하세요.

  ## 🔒 내부 Scratchpad (절대 출력 금지)
  **ReAct Loop (Reason → Act) — 최대 2회 반복**  
  - *Reason*: 질문 유형·감정 파악 → 필요한 데이터 확인  
  - *Act*   : (a) 정보 조회, (b) 파이썬 계산, (c) 추가 질문 중 택1  
  종료 조건 ▲ 정보 충분 → ToT & PAL 단계 진입

  **Tree-of-Thoughts**  
  - ① 주소 문제 ② 배송 지연 ③ 운송장 오류  각각 해결 경로 제시  
  - 비용·날짜 등은 scratchpad기반 Python 코드로 계산  
  - 최적 경로 선택(근거 1줄 기록)

  **Automatic Prompt Engineering**  
  - 초안 작성 후 중복·군더더기 제거하여 100~120 단어로 압축  
  - Meta-Prompt: “<compress/>” 토큰 이후 자체 요약 수행

  **Length / Token 관리**  
  - 응답 ≤120 단어, 4줄 이내, 숫자 목록 사용  
  - 과거 대화는 `sliding_ctx` 요약만 참고 (Max-Token-Bypass)

  **Ask-for-Context**  
  - 필수 정보 비어 있으면 ➜ “추가 정보 요청”만 출력하고 ReAct 종료

  ## ✅ 외부 출력 포맷
  1. 고객명·상품·현재 상태 + 공감 문구 (1줄)  
  2. 조치 계획·예상 일정 (숫자 목록 최대 3개)  
  3. “추가 문의사항이 있으면 언제든 말씀해주세요.” (1줄)

assistant: |-
  {% if history_summary -%}
  🔄 이전 대화 요약: {{history_summary}}
  {%- endif %}
  {% if sliding_ctx -%}
  🔗 최근 맥락: {{sliding_ctx}}
  {%-

ERROR:langfuse:Error while fetching prompt 'order_delivery-label:production': status_code: 404, body: {'message': "Prompt not found: 'order_delivery' with label 'production'", 'error': 'LangfuseNotFoundError'}


✅ Prompt created (v4.0)
👀 Langfuse UI ▸ Prompts ▸ order_delivery 확인


## 작업한 V1.0 Prompty 파일 불러와서, 시나리오 결과 돌리기.

In [14]:
from jinja2 import Template

def render_prompt(messages: list, variables: dict) -> list:
    """Langfuse prompt template (list of dicts) → rendered OpenAI messages"""
    rendered = []
    for message in messages:
        role = message["role"]
        content_template = message["content"]
        content = Template(content_template).render(**variables)
        rendered.append({"role": role, "content": content})
    return rendered


In [15]:
"""
• Scenario_QA.csv → 10건 Async 처리(gpt-4o-mini)
• 프롬프트: order_delivery/v4_0@stable (smart_cs)
• 결과: data/01_order_delivery/answer_results/Scenario_QA_V4_gpt-4o-mini_<ts>.xlsx
"""
import asyncio, time
from datetime import datetime
from pathlib import Path
import nest_asyncio, pandas as pd
from langfuse import Langfuse
from openai import AsyncOpenAI
from langfuse.decorators import langfuse_context
from langfuse.decorators import observe

nest_asyncio.apply()

# ─── 경로 세팅 ────────────────────────────────────────────────
# VSCODE
# BASE = Path("../data/01_order_delivery")
# COLAB
BASE = Path("/content/drive/MyDrive/KAKAO - 01 - 고급 프롬프트 엔지니어링/kakao_01_advanced_prompt_engineering/data/01_order_delivery")

RESULT_DIR = BASE / "answer_results"
RESULT_DIR.mkdir(exist_ok=True)

# ─── Langfuse / Prompt ──────────────────────────────────────
lf  = Langfuse()
PROMPT = lf.get_prompt("order_delivery", label="stable").prompt  # <-- 레이블 lookup

# ─── CSV 로딩 ────────────────────────────────────────────────
scenario = pd.read_csv(BASE / "Scenario_QA.csv")
cust     = pd.read_csv(BASE / "Customer_Info.csv")
addr     = pd.read_csv(BASE / "Delivery_Address.csv")
order    = pd.read_csv(BASE / "Order_Info.csv")
shipping = pd.read_csv(BASE / "Shipping_Issue_Log.csv")

df = (
    scenario
    .merge(cust,  on="customer_id", suffixes=("", "_cust"), how="left")
    .merge(order, on="customer_id", suffixes=("", "_order"), how="left")
    .merge(addr, on="customer_id", suffixes=("", "_addr"), how="left")
    .merge(shipping, on="order_id", suffixes=("", "_shipping"), how="left")
)

# ─── LLM 호출 ───────────────────────────────────────────────
MODEL  = "gpt-4o-mini"
client = AsyncOpenAI()  # OPENAI_API_KEY 환경변수 필요

@observe()
async def call_llm(row):
    prompt_input = {
        "question":          row.question,
        "customer_id":       row.customer_id,
        "customer_name":     row.customer_name,
        "order_id":          row.order_id,
        "product_name":      row.product_name,
        "shipping_status":   row.shipping_status,
        "last_update":       row.last_update or "",
        "shipping_company":  row.shipping_company or "",
        "tracking_number":   row.tracking_number or "",
        "address_line1":     row.address_line1,
        "city":              row.city,
        "postal_code":       row.postal_code,
    }

    # Langfuse trace (session metadata)
    langfuse_context.update_current_trace(
        name       = "order_delivery",
        user_id    = row.customer_id,
        session_id = row.scenario_id,
        tags       = ["V4", "smart_cs"],
        metadata   = {"model": MODEL},
    )

    # Langfuse Prompt 템플릿 메시지 → 실제 messages 생성
    rendered_messages = render_prompt(PROMPT, prompt_input)

    start = time.perf_counter_ns()

    # 직접 OpenAI 호출
    response = await client.chat.completions.create(
        model       = MODEL,
        messages    = rendered_messages,
        temperature = 0.3,
        max_tokens  = 350,
    )

    latency_ms = (time.perf_counter_ns() - start) / 1e6

    return response.choices[0].message.content, latency_ms, response.usage.prompt_tokens, response.usage.completion_tokens

async def main():
    tasks   = [call_llm(row) for _, row in df.iterrows()]
    results = await asyncio.gather(*tasks)

    out = df.copy()
    out[["answer", "latency_ms", "prompt_tokens", "completion_tokens"]] = pd.DataFrame(results)

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = RESULT_DIR / f"Scenario_QA_V4_gpt-4o-mini_{ts}.xlsx"
    out.to_excel(out_path, index=False)
    print(f"✅ 결과 저장: {out_path}")


asyncio.run(main())


✅ 결과 저장: /content/drive/MyDrive/KAKAO - 01 - 고급 프롬프트 엔지니어링/kakao_01_advanced_prompt_engineering/data/01_order_delivery/answer_results/Scenario_QA_V4_gpt-4o-mini_20250616_212808.xlsx
